#  T1 and T2 Time

In [ ]:
import matplotlib.pyplot as plt
from equal1.benchmarking.device_characterization import thermal_noise
import pandas as pd
import seaborn as sns
from tqdm import tqdm

In [ ]:
hardware = {
    "hpc-conservative": 32,
    "hpc-gamma1": 32,
    "hpc-gamma2": 32,
}

In [ ]:
all_experiments_t1 = [
    thermal_noise.IdleNoiseT1(device_name=device, qubit_idx=qubit)
    for device, num_qubits in hardware.items()
    for qubit in range(num_qubits)
]

all_experiments_t2 = [
    thermal_noise.ThermalNoiseT2Ramsey(device_name=device, qubit_idx=qubit)
    for device, num_qubits in hardware.items()
    for qubit in range(num_qubits)
]

In [ ]:
results = []

for t1_experiment, t2_experiment in tqdm(zip(all_experiments_t1, all_experiments_t2)):
    assert t1_experiment.device_name == t2_experiment.device_name
    assert t1_experiment.qubit_idx == t2_experiment.qubit_idx

    device_name = t1_experiment.device_name
    qubit_idx = t1_experiment.qubit_idx

    t1_experiment.run(
        simulate=True,
        runtime_options={
            "backend": "TensorNet_MPS",
            "simulation_platform": "GPU",
        },
    )
    t1 = t1_experiment.analyse_results()

    t2_experiment.run(
        simulate=True,
        runtime_options={
            "backend": "TensorNet_MPS",
            "simulation_platform": "GPU",
        },
    )
    t2 = t2_experiment.analyse_results()

    results.append(
        {
            "device": device_name,
            "qubit_idx": qubit_idx,
            "t1": t1,
            "t2": t2,
        }
    )

In [ ]:
df = pd.DataFrame(results)

device_groups = {device: group for device, group in df.groupby("device")}

for device, group in device_groups.items():
    group.to_csv(f"t1_t2_results_{device}.csv", index=False)

In [ ]:
for device, group in device_groups.items():
    fig, ax = plt.subplots(figsize=(24, 6))
    sns.barplot(data=group, x="qubit_idx", y="t1", ax=ax)
    ax.set_title(f"T1 Time by Qubit - {device}")
    ax.set_ylabel("T1 Time (s)")
    ax.set_xlabel("Qubit Index")

In [ ]:
for device, group in device_groups.items():
    fig, ax = plt.subplots(figsize=(24, 6))
    sns.barplot(data=group, x="qubit_idx", y="t2", ax=ax)
    ax.set_title(f"T2 Time by Qubit - {device}")
    ax.set_ylabel("T2 Time (s)")
    ax.set_xlabel("Qubit Index")